
# --Topic: Toxic Comment Classifier--

# Sử dụng các mô hình học máy để dự đoán khả năng của đoạn văn (comment_text) theo 6 trường dữ liệu : 
* Toxic
* Severe Toxic
* Obscene
* Insult 
* Threat
* Identity Hate

# Mục Lục
# 1. Tổng quan
# 2. Thực hiện
# 3. Đánh giá và kết luận

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Giải nén data
import zipfile
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
unzip.extractall()

# 1. Tổng quan

# Mô hình dùng để làm gì: 
> Mô hình được tạo ra với mục đích xét khả năng toxic của dữ liệu text mà mình mong muốn, to lớn hơn là giúp cho không gian mạng sạch sẽ hơn rất nhiều

In [ ]:
#Khởi chạy thử data cần train và nhận xét

train_df = pd.read_csv('/kaggle/working/train.csv')# Loading data
train_df.head() #show data

# Nhận xét tổng quan về dữ liệu: Dữ liệu sử dụng để training chưa được tối ưu hóa trong trường comment_text, cần có bước làm sạch dữ liệu trước khi training

In [ ]:
#Khởi chạy thử data cần test

test_df = pd.read_csv('/kaggle/working/test.csv')# Loading data
test_df.head() #show data

In [ ]:
import nltk
from nltk.corpus import stopwords  # Xóa ký tự không cần thiết
from nltk.stem.lancaster import LancasterStemmer # convert words to base form

In [ ]:
# Tải stopword trong thư viện nltk
nltk.download('stopwords')

In [ ]:
set(stopwords.words('english')) # Chon English

# Chúng ta sử dụng stopword để làm gì?
- Vì trong một câu có những từ có tần số xuất hiện nhiều như the, to... các từ này thường mang ít giá trị ý nghĩa và không khác nhau nhiều trong các văn bản khác nhau. 
- Ví dụ từ "the" hay "to" thì ở văn bản nào nó cũng không bị thay đổi về ý nghĩa.
- Vì thế chúng ta có thể xóa bỏ những từ này

In [ ]:
# Hiển thị thử info của data
train_df.info()

In [ ]:
train_df['comment_text'][0]

In [ ]:
train_df['comment_text'][1]

# Như đã nói ở trên, comment_text chưa được xử lý trước khi training 

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị
train_df.toxic.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị 
train_df.severe_toxic.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị
train_df.obscene.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị
train_df.insult.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị
train_df.threat.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
#Trả về một Chuỗi chứa số lượng các giá trị
train_df.identity_hate.value_counts(normalize=True)
# normalize=True thì đối tượng được trả về sẽ chứa các tần số tương đối của các giá trị

In [ ]:
train_df.head()

In [ ]:
#tính tổng các giá trị lấy từ cột thứ 3
data_count=train_df.iloc[:,2:].sum() 
data_count

# Thứ tự tăng dần là:
- threat
- identity_hate
- severe_toxic
- insult
- obscene
- toxic

# Hiển thị thử các trường hợp thành một biểu đồ cột

In [ ]:
import matplotlib.pyplot as plt
import nltk
import re
import string
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
#sử dụng plot biểu đồ
plt.figure(figsize=(8,4))

#Sử dụng phương thức barplot trong Seaborn
#Hiển thị ước tính điểm và khoảng tin cậy dưới dạng thanh hình chữ nhật.
ax = sns.barplot(data_count.index, data_count.values, alpha=0.8)

plt.title("Bảng giá trị") # đặt tên biểu đồ
plt.ylabel("", fontsize=12)
plt.xlabel("Loại", fontsize=12) # đặt tên cho trục hoành , set font là 12

#Thêm text cho mỗi cột
rects = ax.patches
labels = data_count.values
for rect, label in zip(rects, labels):
  height = rect.get_height()
  ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

# Thứ tự tăng dần là:
- threat
- identity_hate
- severe_toxic
- insult
- obscene
- toxic
# Nhìn vào biểu đồ có thể thấy toxic là trường có nhiều nhất

In [ ]:
#lấy độ dài của data
num_rows=len(train_df)
print(num_rows)

# Xét thử phần trăm của các trường so với tổng độ dài Data

In [ ]:
#Create bar graph
sum_tox = train_df['toxic'].sum() / num_rows * 100
sum_sev = train_df['severe_toxic'].sum() / num_rows * 100
sum_obs = train_df['obscene'].sum() / num_rows * 100
sum_thr = train_df['threat'].sum() / num_rows * 100
sum_ins = train_df['insult'].sum() / num_rows * 100
sum_ide = train_df['identity_hate'].sum() / num_rows * 100

ind = np.arange(6)

ax = plt.barh(ind, [sum_tox, sum_sev, sum_obs, sum_thr, sum_ins, sum_ide])
plt.xlabel('Percentage (%)', size=20)
plt.xticks(np.arange(0, 30, 5), size=20)
plt.yticks(ind, ('Toxic', 'Severe Toxic', 'Obscene', 'Threat', 'Insult', 'Identity Hate'), size=15)

plt.gca().invert_yaxis()
plt.show()

# Ta điều chỉnh lại 1 chút cho dễ nhìn nhận hơn

In [ ]:
#Create bar graph
sum_tox = train_df['toxic'].sum() / num_rows * 100
sum_sev = train_df['severe_toxic'].sum() / num_rows * 100
sum_obs = train_df['obscene'].sum() / num_rows * 100
sum_thr = train_df['threat'].sum() / num_rows * 100
sum_ins = train_df['insult'].sum() / num_rows * 100
sum_ide = train_df['identity_hate'].sum() / num_rows * 100

ind = np.arange(6)

ax = plt.barh(ind, [sum_tox, sum_obs, sum_ins,  sum_sev, sum_ide , sum_thr])
plt.xlabel('Percentage (%)', size=20)
plt.xticks(np.arange(0, 30, 5), size=20)
plt.yticks(ind, ('Toxic', 'Obscene', 'Insult', 'Severe Toxic', 'Identity Hate', 'Threat'), size=15)

plt.gca().invert_yaxis()
plt.show()

- Có tận 3 trường là dưới 5%
- 2 trường Obscene và Insult khoảng 5%
- Và trường Toxic vượt quá 10%

# 2. Thực thi

# Giai đoạn tiền xử lý dữ liệu

In [ ]:
#Tiền xử lý
# xóa tất cả các số có chữ cái gắn liền với chúng
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)

# thay thế dấu câu bằng khoảng trắng
#convert tất cả chuỗi thành chữ thường
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

#xóa tất cả '\n'
remove_n = lambda x:re.sub("\n", " ", x)

#xóa các ký tự không phải ascii
remove_non_ascii = lambda x: re.sub(r'[^\x00-\x7f]',r' ',x)

#Apply map
train_df['comment_text'] = train_df['comment_text'].map(alphanumeric).map(punc_lower).map(remove_n).map(remove_non_ascii)

#Show comment_text 0
train_df['comment_text'][0]

# - So sánh với comment_text[0] trước đó:
- "Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

# - Có thể thấy, các dấu đã bị bỏ đi, các số , ký tự "/n" đã bị loại bỏ và chuyển về dạng viết thường hết

In [ ]:
# Chia thanh 6 section
data_tox = train_df.loc[:,['id', 'comment_text', 'toxic']]
data_sev = train_df.loc[:,['id', 'comment_text', 'severe_toxic']]
data_obs = train_df.loc[:,['id', 'comment_text', 'obscene']]
data_ins = train_df.loc[:,['id', 'comment_text', 'insult']]
data_thr = train_df.loc[:,['id', 'comment_text', 'threat']]
data_ide = train_df.loc[:,['id', 'comment_text', 'identity_hate']]

In [ ]:
pip install wordcloud

# Tạo các WordCloud
**Những đám mây chứa rất nhiều từ ở các kích thước khác nhau, chúng thể hiện tần suất hoặc tầm quan trọng của mỗi từ.**

In [ ]:
import wordcloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords

In [ ]:
def wordcloud(df, label):
    subset=df[df[label]==1]
    text=subset.comment_text.values
    wc=WordCloud(background_color='black',max_words=4000)
    
    wc.generate(" ".join(text))
    
    plt.figure(figsize=(20,20))
    plt.subplot(221)
    plt.axis('off')
    plt.title("Words frequented in {}".format(label), fontsize=20)
    plt.imshow(wc.recolor(colormap='gist_earth', random_state=244), alpha=0.98)

In [ ]:
wordcloud(data_tox,'toxic')

In [ ]:
wordcloud(data_ide,'identity_hate')

In [ ]:
wordcloud(data_sev,'severe_toxic')

In [ ]:
wordcloud(data_obs,'obscene')

In [ ]:
wordcloud(data_ins,'insult')

In [ ]:
wordcloud(data_thr,'threat')

In [ ]:
data_tox.head()

# Lập chỉ mục hoàn toàn dựa trên vị trí số nguyên để lựa chọn theo vị trí.

In [ ]:
data_tox_1 = data_tox[data_tox['toxic'] == 1].iloc[0:5000,:]
data_tox_1.shape

In [ ]:
data_tox_0 = data_tox[data_tox['toxic'] == 0].iloc[0:5000,:]

# Nối lại bằng concat
# Axis: Trục nối dọc

In [ ]:
data_tox_done = pd.concat([data_tox_1, data_tox_0], axis = 0)
data_tox_done.shape

# Đếm Severe Toxic 

In [ ]:
data_sev[data_sev['severe_toxic'] == 1].count()

In [ ]:
data_sev_1 = data_sev[data_sev['severe_toxic'] == 1].iloc[0:1595,:]
data_sev_0 = data_sev[data_sev['severe_toxic'] == 0].iloc[0:1595,:]
data_sev_done = pd.concat([data_sev_1, data_sev_0], axis = 0)
data_sev_done.shape

In [ ]:
data_obs[data_obs['obscene'] == 1].count()

In [ ]:
data_obs_1 = data_obs[data_obs['obscene'] == 1].iloc[0:5000,:]
data_obs_0 = data_obs[data_obs['obscene'] == 0].iloc[0:5000,:]
data_obs_done = pd.concat([data_obs_1, data_obs_0], axis = 0)
data_obs_done.shape

In [ ]:
data_thr[data_thr['threat'] == 1].count()

In [ ]:
data_thr_1 = data_thr[data_thr['threat'] == 1].iloc[0:478,:] #20%
data_thr_0 = data_thr[data_thr['threat'] == 0].iloc[0:1912,:]#80%
data_thr_done = pd.concat([data_thr_1, data_thr_0], axis = 0)
data_thr_done.shape

In [ ]:
data_ins[data_ins['insult'] == 1].count()

In [ ]:
data_ins_1 = data_ins[data_ins['insult'] == 1].iloc[0:5000,:]
data_ins_0 = data_ins[data_ins['insult'] == 0].iloc[0:5000,:]
data_ins_done = pd.concat([data_ins_1, data_ins_0], axis = 0)
data_ins_done.shape

In [ ]:
data_ide[data_ide['identity_hate'] == 1].count()

In [ ]:
data_ide_1 = data_ide[data_ide['identity_hate'] == 1].iloc[0:1405,:] #20%
data_ide_0 = data_ide[data_ide['identity_hate'] == 0].iloc[0:5620,:] #80%
data_ide_done = pd.concat([data_ide_1, data_ide_0], axis = 0)
data_ide_done.shape

# Nhắc nhở: Số lượng nhận xét thuộc các danh mục sau:
* Toxic ( 14000+)
* Severe Toxic (1595)
* Obscene (8449)
* Threat (478)
* Insult (7877)
* Identity Hate (1405) 


//-------------------------------------------------------//

# Part 3 : Chạy Mô hình ML trên dữ liệu

In [ ]:
#nhập các gói để xử lý trước
from sklearn import preprocessing
from sklearn.feature_selection import SelectFromModel

#nhập các công cụ để chia nhỏ dữ liệu và đánh giá hiệu suất mô hình
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, fbeta_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve

#import ML algos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Tạo chức năng đơn giản có trong tập dữ liệu và cho phép người dùng chọn tập dữ liệu, nhãn độc tính, vectơ và số lượng ngam

In [ ]:
def cv_tf_train_test(df_done,label,vectorizer,ngram):
    #Chia dữ liệu thành các tập dữ liệu X và y
    X = df_done.comment_text
    y = df_done[label]
    
    #Chia ngày của chúng tôi thành dữ liệu đào tạo và kiểm tra
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
    
    #Tạo một đối tượng Vectorizer và xóa các stopword dừng khỏi bảng
    cv1 = vectorizer(ngram_range=(ngram), stop_words='english')
    
    X_train_cv1 = cv1.fit_transform(X_train) # Học từ điển từ vựng và trả về ma trận tài liệu thuật ngữ
    X_test_sv1 = cv1.transform(X_test) #Học từ điển từ vựng của tất cả các mã thông báo trong tài liệu thô
    
    # Dùng các model để train
    lr = LogisticRegression()
    lr.fit(X_train_cv1, y_train)
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_cv1, y_train)
    
    
    bnb = BernoulliNB()
    bnb.fit(X_train_cv1, y_train)
    
    mnb = MultinomialNB()
    mnb.fit(X_train_cv1, y_train)
    
    svm_model = LinearSVC()
    svm_model.fit(X_train_cv1, y_train)
    
    randomforest = RandomForestClassifier(n_estimators=100, random_state = 42)
    randomforest.fit(X_train_cv1, y_train)
    
    f1_score_data = {'F1 Score':[f1_score(lr.predict(X_test_sv1), y_test), 
                                 f1_score(knn.predict(X_test_sv1), y_test),
                                 f1_score(bnb.predict(X_test_sv1), y_test),
                                 f1_score(mnb.predict(X_test_sv1), y_test), 
                                 f1_score(svm_model.predict(X_test_sv1), y_test),
                                f1_score(randomforest.predict(X_test_sv1), y_test),]}
    
    
    df_f1 = pd.DataFrame(f1_score_data, index=['Log Regression', 'KNN', 'BernoulliNB', 'MultinomialNB', 'SVM', 'Random Forest'])
    return df_f1

# Tạo một Frame hiện thị tất cả những mô hình mà mình vừa sử dụng
# (Nhận xét sau khi tạo biểu đồ đường)

In [ ]:
df_tox_cv = cv_tf_train_test(data_tox_done, 'toxic', TfidfVectorizer, (1,1))
df_tox_cv.rename(columns={'F1 Score': 'F1 Score(toxic)'}, inplace=True)
df_tox_cv

In [ ]:
df_sev_cv = cv_tf_train_test(data_sev_done, 'severe_toxic', TfidfVectorizer, (1,1))
df_sev_cv.rename(columns={'F1 Score': 'F1 Score(Severe toxic)'}, inplace=True)
df_sev_cv

In [ ]:
df_obs_cv = cv_tf_train_test(data_obs_done, 'obscene', TfidfVectorizer, (1,1))
df_obs_cv.rename(columns={'F1 Score': 'F1 Score(Obscene)'}, inplace=True)
df_obs_cv

In [ ]:
df_ins_cv = cv_tf_train_test(data_ins_done, 'insult', TfidfVectorizer, (1,1))
df_ins_cv.rename(columns={'F1 Score': 'F1 Score(Insult)'}, inplace=True)
df_ins_cv

In [ ]:
df_thr_cv = cv_tf_train_test(data_thr_done, 'threat', TfidfVectorizer, (1,1))
df_thr_cv.rename(columns={'F1 Score': 'F1 Score(Threat)'}, inplace=True)
df_thr_cv

In [ ]:
df_ide_cv = cv_tf_train_test(data_ide_done, 'identity_hate', TfidfVectorizer, (1,1))
df_ide_cv.rename(columns={'F1 Score': 'F1 Score(Identity Hate)'}, inplace=True)
df_ide_cv

# Cùng xem xét lại thành 1 bảng hoàn chỉnh

In [ ]:

f1_all = pd.concat([df_tox_cv, df_sev_cv, df_obs_cv, df_ins_cv, df_thr_cv, df_ide_cv], axis=1)
f1_all

# Thay đổi góc nhìn 1 chút ta được bảng sau:

In [ ]:
f1_all_trp = f1_all.transpose() #Trả về chế độ xem của mảng có các trục được hoán vị.
f1_all_trp

# Tạo một đồ thị đường để so sánh các giá trị mới tìm được:

In [ ]:
sns.lineplot(data=f1_all_trp, markers=True)
#sns.relplot(data=flights, x="year", y="passengers", hue="month", kind="line")
plt.xticks(rotation='90', fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='best')
plt.title('F1 Score', fontsize=20)

# Từ các bảng và biểu đồ trên có thể thấy được
* Random Forest và SVM có độ ổn định và đánh giá cao hơn hẳn
* KNN và BernoulliNB không cao lắm
# ==>Sẽ dùng RandomForest để dự đoán

In [ ]:
#Xây dựng lại 1 hàm để huấn luyện data
def train_test(df_done, label, vectorizer, ngram):
    X = df_done.comment_text
    y = df_done[label]
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    cv1 = vectorizer(ngram_range=(ngram), stop_words='english')
    
    X_train_cv1 = cv1.fit_transform(X_train) 
    print(X_train_cv1)
    X_test_cv1 = cv1.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_cv1, y_train)
    return randomforest.predict(X_test_cv1)

# Chuyển đổi một bộ sưu tập các tài liệu thô thành một ma trận các tính năng TF-IDF.

In [ ]:
df_tox = train_test(data_tox_done,'toxic', TfidfVectorizer, (1,1))
df_tox

In [ ]:
df_sev = train_test(data_sev_done,'severe_toxic', TfidfVectorizer, (1,1))
df_sev

In [ ]:
df_obs = train_test(data_obs_done,'obscene', TfidfVectorizer, (1,1))
df_obs

In [ ]:
df_ins = train_test(data_ins_done,'insult', TfidfVectorizer, (1,1))
df_ins

In [ ]:
df_thr = train_test(data_thr_done,'threat', TfidfVectorizer, (1,1))
df_thr

In [ ]:
df_ide = train_test(data_ide_done,'identity_hate', TfidfVectorizer, (1,1))
df_ide

//---------------------------------------------//

# Bây giờ sẽ dự đoán thử khả năng Toxic của một câu

In [ ]:
    X = data_tox_done.comment_text
    y = data_tox_done['toxic']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)

# Một function dự đoán khả năng toxic

In [ ]:
#Predict Toxic Function 
def predictToxic(sample):
    vect = tfv.transform(sample)
    return randomforest.predict_proba(vect)[:,1:]

# Dự đoán khả năng củ từ "Fuck you Nigga"

In [ ]:
print('Du doan cua Toxic: ', predictToxic(['Fuck you nigga']))

> Khá là chính xác


# Dự đoán từ "I Love You"

In [ ]:
print('Du doan cua Toxic: ', predictToxic(['I Love You']))

> Không có vấn đề gì cả, dưới 0.5

# Dự đoán từ"How are you today"

In [ ]:
print('Du doan cua Toxic: ', predictToxic(['How are you today']))

# Bây giờ sẽ dự đoán thử khả năng Severe Toxic của một câu

In [ ]:
    X = data_sev_done.comment_text
    y = data_sev_done['severe_toxic']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)

# Severe Toxic Function

In [ ]:
#Predict Severe Toxic Function 
def predict(sample):
    vect = tfv.transform(sample)
    return randomforest.predict_proba(vect)[:,1:]

In [ ]:
print('Du doan cua Severe Toxic: ', predict(['Fuck you nigga']))

In [ ]:
print('Du doan cua Severe Toxic: ', predict(['I love you']))

**Severe Toxic khá là chính xác**

# Dự đoán obscene

In [ ]:
    X = data_obs_done.comment_text
    y = data_obs_done['obscene']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)
    
    #Predict Obscene Function 
    def predict(sample):
        vect = tfv.transform(sample)
        return randomforest.predict_proba(vect)[:,1:]

    print('Du doan cua Obscene: ', predict(['Fuck you nigga']))
    print('Du doan cua Obscene: ', predict(['Ilove you']))

**Chính xác tuyệt đối**

# Dự đoán Insult 

In [ ]:
    X = data_ins_done.comment_text
    y = data_ins_done['insult']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)
    
    #Predict Obscene Function 
    def predict(sample):
        vect = tfv.transform(sample)
        return randomforest.predict_proba(vect)[:,1:]

    print('Du doan cua Insult: ', predict(['Fuck you nigga']))
    print('Du doan cua Insult: ', predict(['Ilove you']))

**Gần như là hoàn hảo**

# Dự đoán Threat

In [ ]:
    X = data_thr_done.comment_text
    y = data_thr_done['threat']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)
    
    #Predict Obscene Function 
    def predict(sample):
        vect = tfv.transform(sample)
        return randomforest.predict_proba(vect)[:,1:]

    print('Du doan cua Threat: ', predict(['Fuck you nigga']))
    print('Du doan cua Threat: ', predict(['Ilove you']))

**Lỗi**

# Dự đoán identity Hate

In [ ]:
    X = data_ide_done.comment_text
    y = data_ide_done['identity_hate']
    
    #chia thành các phần tử con để thực hiện huấn luyện
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    #Tạo Vectorizer object và xóa stopwords khỏi bảng
    tfv = TfidfVectorizer(ngram_range=(1,1), stop_words='english')
    
    X_train_fit = tfv.fit_transform(X_train) 
    X_test_fit = tfv.transform(X_test)
    
    #Chọn mô hình Random Forest
    randomforest = RandomForestClassifier(n_estimators=100, random_state=42)
    randomforest.fit(X_train_fit, y_train)
    randomforest.predict(X_test_fit)
    
    #Predict Obscene Function 
    def predict(sample):
        vect = tfv.transform(sample)
        return randomforest.predict_proba(vect)[:,1:]

    print('Du doan cua Identity Hate: ', predict(['Fuck you nigga']))
    print('Du doan cua Identity Hate: ', predict(['Ilove you']))

# Như vậy là chúng ta đã test thử với 2 câu
* "Fuck you nigga" 
* I Love You 
# Có thể thấy, kết quả khá là cao nhưng vẫn còn trường threat là hơi sai 1 chút nhưng nhìn chung là mô hình của chúng ta khá hoàn chỉnh với RandomForest

# -------------------------------------------

# Bây giờ chúng ta cùng nói về thuật toán mình chọn trong bài toán 
# Random Forest là gì và tại sao nó lại tốt
1. Random = Tính ngẫu nhiên
2. Forest = Rừng = Nhiều cây quyết định

**- Random forest là một trong những thuật toán học máy mạnh mẽ và phổ biến nhất, nó là thuật toán supervised learning, có thể giải quyết cả bài toán regression và classification.**
**Random Forest là sự cải tiến của bagging. Nó sử dụng các cây (tree) để làm nền tảng, là một tập hợp của hàng trăm cây quyết định, trong đó mỗi cây được tạo nên ngẫu nhiên từ việc tái chọn mẫu (chọn random 1 phần của dữ liệu để xây dựng) và random các đặc trưng (feature) từ toàn bộ dữ liệu.**
![](https://www.freecodecamp.org/news/content/images/2020/08/how-random-forest-classifier-work.PNG)

**- Ví dụ cho dễ hiểu:
Bạn muốn đi mua một thứ gì đó nhưng bạn muốn có được những thứ tốt nhất, bạn phải cân nhắc địa điểm mua hàng cho nên, bạn phải tham khảo ý kiến của nhiều nơi khác nhau. Mỗi một ý kiến ở đây sẽ đóng vai trò như một CÂY QUYẾT ĐỊNH trả lời cho những câu hỏi của bạn. Rồi sau đó bạn sẽ có một loạt câu trả lời cho những câu hỏi của bạn, từ đó chọn được phương án tốt nhất.**


**Random Forest hoạt động cũng như thế, mỗi cây quyết định được xây dựng dùng thuật toán Decision Tree trên tập dữ liệu khác nhau và dùng tập thuộc tính khác nhau. Sau đó bằng cách đánh giá các cây quyết định sử dụng cách thức voting để đưa ra kết quả cuối cùng cho bài toán**

**Nếu thuật toán Random Forest có 6 cây quyết đinh, 5 cây dự đoán 1, 1 cây dự đoán 0, do đó mình sẽ lấy và cho ra dự đoán cuối cùng là 1**

# Ưu và nhược điểm của thuật toán Random Forest 

1. Ưu điểm 
* Thuật toán dễ sử dụng và mạnh mẽ
* Giảm phương án sai, tránh bị Overfitting
* Có thể sử dụng cho cả 2 loại bài toán là 
* Nó có thể làm được với những bài toán bị thiếu dữ li

2. Nhược điểm
* Mất khả năng diễn giải của mô hình
* Bagging rất mạnh, cho chúng ta độ chính xác cao hơn, nhưng lại nặng về mặt tính toán và có thể có trường hợp không có được kết quả mong muốn giống như trường threat của chúng ta ở trên.